In [1]:
import os

from transformers.utils import is_torch_fx_proxy

# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"  # see issue #152
# os.environ["CUDA_VISIBLE_DEVICES"] = "2"

import datasets
import torch
from torch import nn
from transformers import MBartModel, MBartTokenizer, MBartConfig, MBartForConditionalGeneration
from datasets import concatenate_datasets, load_from_disk
from torch.optim import *
from torch.utils import data
from torch.utils.data import DataLoader, TensorDataset
from datasets import load_dataset

In [2]:
def model_size(model):
    param_size = 0
    for param in model.parameters():
        param_size += param.nelement() * param.element_size()
    buffer_size = 0
    for buffer in model.buffers():
        buffer_size += buffer.nelement() * buffer.element_size()

    size_all_mb = (param_size + buffer_size) / 1024 ** 2
    return size_all_mb

In [3]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

#pre_train_ds = load_dataset("text", data_files={"train": ["/data/n.dallanoce/cc100/en.txt"]},
#                            cache_dir="/data/n.dallanoce/cc100/hugg_en", split='train', ignore_verifications=True)
tok_name = "thesistranslation/mbart_en_fr"
tok_en = AutoTokenizer.from_pretrained(tok_name, src_lang="en_XX")
cuda_dev = "cpu"
#model = model.to(cuda_dev)
model: MBartForConditionalGeneration = MBartForConditionalGeneration.from_pretrained(
    "/data/n.dallanoce/weights/mt5_ft_en-fr_M1F1_t5_mbart/checkpoint-5000").to(cuda_dev)
model.train(False)
print(model_size(model))

#dataset_loaded = load_from_disk("europarl_eng_tokenized")
#pre_train_load = DataLoader(pre_train_ds, batch_size=8, drop_last=True, shuffle=False, pin_memory=True, num_workers=4)

235.07921600341797


In [30]:
print(list(model.modules()))

[MBartForConditionalGeneration(
  (model): MBartModel(
    (shared): Embedding(32030, 512, padding_idx=1)
    (encoder): MBartEncoder(
      (embed_tokens): Embedding(32030, 512, padding_idx=1)
      (embed_positions): MBartLearnedPositionalEmbedding(1026, 512)
      (layers): ModuleList(
        (0): MBartEncoderLayer(
          (self_attn): MBartAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=512, out_features=2048, bias=True)
          (fc2): Linear(in_features=2048, out_features=512, bias=True)
          (final_layer_norm): LayerNorm((512,), eps

In [1]:
model.model.encoder.layers[0]

NameError: name 'model' is not defined

In [28]:
param_lst = list(model.model.get_submodule("encoder.layers.0.self_attn.k_proj").parameters())
print([e.shape for e in param_lst])
print(param_lst)

[torch.Size([512, 512]), torch.Size([512])]
[Parameter containing:
tensor([[-0.0552, -0.0768,  0.5730,  ...,  0.0457,  0.3106,  0.0737],
        [-0.2644, -0.0690,  0.0978,  ...,  0.2587, -0.2191,  0.2115],
        [ 0.0304, -0.3047,  0.3530,  ...,  0.1265,  0.2673, -0.0974],
        ...,
        [-0.0083, -0.0276,  0.0968,  ...,  0.3071, -0.0715, -0.0855],
        [-0.5523,  0.2412, -0.0553,  ..., -0.1232,  0.0274,  0.1091],
        [ 0.1764, -0.0348, -0.1450,  ..., -0.2204,  0.1946,  0.0720]],
       device='cuda:0', requires_grad=True), Parameter containing:
tensor([ 1.1361e-02, -9.2545e-02,  1.2006e-01,  9.9594e-03, -9.1281e-02,
        -2.9175e-02,  1.0413e-01,  2.1937e-01, -1.2781e-01,  1.3878e-02,
        -4.9402e-03,  1.7313e-01,  7.9461e-05,  1.8383e-01,  9.9675e-02,
        -1.4741e-02, -2.9353e-01,  6.7065e-02,  1.8205e-01,  2.6332e-02,
        -9.2180e-02,  8.1779e-02, -2.9540e-02,  1.1995e-01,  1.8370e-01,
         9.8391e-02, -1.3431e-01, -1.1188e-01,  1.8217e-01,  1.0096

In [25]:
with torch.no_grad():
    print(torch.nn.functional.gelu(param_lst[0]))

torch.Size([2048, 512])


In [21]:
sent = "What are you doing for the session?</s> How are you?"
out = tok_en(sent, add_special_tokens=True, return_tensors="pt")
gen = model.generate(out['input_ids'].to(cuda_dev), num_beams=5, max_length=128,
                     decoder_start_token_id=tok_en.convert_tokens_to_ids("fr_XX"))
tok_en.batch_decode(gen)

['fr_XX Que faites-vous pour la session?</s>']

In [10]:
model.model.shared.weight

Parameter containing:
tensor([[ 0.1152, -0.1643,  0.1987,  ...,  0.1372, -0.0425, -0.2086],
        [ 0.1267, -0.1585,  0.1778,  ...,  0.1125, -0.0720, -0.2186],
        [-0.1357, -0.1363,  0.0593,  ...,  0.0152, -0.0454, -0.2686],
        ...,
        [ 0.1230, -0.1556,  0.1748,  ...,  0.1078, -0.0689, -0.2171],
        [ 0.1261, -0.1566,  0.1794,  ...,  0.1139, -0.0656, -0.2184],
        [ 0.1245, -0.1576,  0.1732,  ...,  0.1074, -0.0787, -0.2140]],
       requires_grad=True)

In [11]:
model.lm_head.weight

Parameter containing:
tensor([[ 0.1152, -0.1643,  0.1987,  ...,  0.1372, -0.0425, -0.2086],
        [ 0.1267, -0.1585,  0.1778,  ...,  0.1125, -0.0720, -0.2186],
        [-0.1357, -0.1363,  0.0593,  ...,  0.0152, -0.0454, -0.2686],
        ...,
        [ 0.1230, -0.1556,  0.1748,  ...,  0.1078, -0.0689, -0.2171],
        [ 0.1261, -0.1566,  0.1794,  ...,  0.1139, -0.0656, -0.2184],
        [ 0.1245, -0.1576,  0.1732,  ...,  0.1074, -0.0787, -0.2140]],
       requires_grad=True)

In [12]:
model.num_parameters()

61592576

In [ ]:
param_tot = 0
for param in model.parameters():
    param_tot += param.nelement()
print(param_tot)

In [ ]:
sentence = "Travelling alone can be a daunting prospect, not least to attend a wedding alone. The original plan had been to go with another neighbour, but she slipped a disc in her back and was unable to fly."
test_ids = tok_en([sentence], add_special_tokens=True, return_tensors="pt")["input_ids"]
logits = model(test_ids.to(cuda_dev)).logits
masked_index = (test_ids[0] == tok_en.mask_token_id)  #.nonzero().item()
masked_index = torch.nonzero(masked_index).item()
probs = logits[0, masked_index].softmax(dim=0)
values, predictions = probs.topk(5)
tok_en.decode(predictions).split()

In [ ]:
example_english_phrase = "Travelling alone can be a daunting prospect, not least to attend a wedding alone. The original plan had been to go with another neighbour, but she slipped a disc in her back and was unable to fly."
batch = tok_en(example_english_phrase, return_tensors="pt").to(cuda_dev)
print(batch["input_ids"])
generated_ids = model.generate(batch["input_ids"], max_new_tokens=128,
                               pad_token_id=tok_en.pad_token_id,
                               eos_token_id=tok_en.eos_token_id)
print(tok_en.batch_decode(generated_ids, skip_special_tokens=True))

In [ ]:
example_english_phrase = "Loving my <mask> almost due can't wait to see my baby boy."
batch = tok_en(example_english_phrase, return_tensors="pt").to(cuda_dev)
print(batch["input_ids"])
generated_ids = model.generate(batch["input_ids"], max_new_tokens=128,
                               pad_token_id=tok_en.pad_token_id,
                               eos_token_id=tok_en.eos_token_id,
                               decoder_start_token_id=tok_en.lang_code_to_id[tok_en.src_lang])
print(tok_en.batch_decode(generated_ids, skip_special_tokens=True))

In [ ]:
cc100_fr = load_dataset("cc100", lang="fr",
                        cache_dir="/data/n.dallanoce/cc100/huggingface",
                        split=f"train[:1024]",
                        verification_mode='no_checks')
cc100_fr[5]

In [ ]:
tok_fr = AutoTokenizer.from_pretrained(tok_name, src_lang="fr_XX")
sent = "Je penses et réfléchit tout seul sur des sujets n'ayant rien à voir avec une situation. Exemple, quand j'attends mes soeurs et ma mère devant un magasin, je réfléchit à la création du monde."
batch = tok_fr(sent, return_tensors="pt").to(cuda_dev)
print(batch["input_ids"])
generated_ids = model.generate(batch["input_ids"], max_new_tokens=128,
                               pad_token_id=tok_fr.pad_token_id,
                               eos_token_id=tok_fr.eos_token_id,
                               decoder_start_token_id=tok_fr.lang_code_to_id[tok_fr.src_lang])
print(tok_en.batch_decode(generated_ids, skip_special_tokens=True))

In [ ]:
tok_fr.lang_code_to_id[tok_fr.src_lang]